In [1]:

import SimpleITK as sitk
import tkinter as tk
from tkinter import filedialog
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
from PIL import Image
import scipy
import os
import shutil
import pandas as pd
from scipy import signal

import sys
# Add the directory to the sys.path
sys.path.append('D:/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis')
# Now you can import modules from that directory
import tools.CT_modification_functions as CT_modification_functions
import tools.CT_visualization_functions as CT_visualization_functions
import tools.CT_visualization_window as CT_visualization_window
import tools.loading_bar as loadingbar
%matplotlib qt

CT_visualization_window.check_existence()

you loaded the CT visualizer window v20.11.24


In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
gr_parent_dir = os.path.dirname(parent_dir)
gr_gr_parent_dir = os.path.dirname(gr_parent_dir)

print(current_dir)
print(parent_dir)
print(gr_parent_dir)
print(gr_gr_parent_dir)



direction_multimatrial_map = current_dir + '\map_multimaterial.mhd'

direction_original_images =  gr_gr_parent_dir + "\\02 - Images Processing\\023 - filtering\\filtered_image-set2.mhd"

output_folder_path = current_dir


d:\COFO - HP\01 PhD research\202403 - Bern skull\WP5 - bern-001\03 - Simulation Setup\031 - create mesh volume\python
d:\COFO - HP\01 PhD research\202403 - Bern skull\WP5 - bern-001\03 - Simulation Setup\031 - create mesh volume
d:\COFO - HP\01 PhD research\202403 - Bern skull\WP5 - bern-001\03 - Simulation Setup
d:\COFO - HP\01 PhD research\202403 - Bern skull\WP5 - bern-001


<>:13: SyntaxWarning: invalid escape sequence '\m'
<>:13: SyntaxWarning: invalid escape sequence '\m'
C:\Users\cofo\AppData\Local\Temp\ipykernel_20756\1109126467.py:13: SyntaxWarning: invalid escape sequence '\m'
  direction_multimatrial_map = current_dir + '\map_multimaterial.mhd'


In [3]:
original_images = sitk.ReadImage(direction_original_images)

images = sitk.ReadImage(direction_multimatrial_map)
images_array = np.array(sitk.GetArrayFromImage(images))
CT_visualization_window.open_window(sitk.GetImageFromArray(images_array))

CT_visualization_functions.browse_blended_images(original_images,images,True)


2024-11-27 10:27:32,944 - CT_window_logger - INFO - starting window setup function
2024-11-27 10:28:35,196 - CT_window_logger - INFO - window setup function succesfull


In [6]:
import cv2

images = sitk.ReadImage(direction_multimatrial_map)
images_array = np.array(sitk.GetArrayFromImage(images)).astype(np.uint8)


def detect_crack(images):
    print("starting 3D convolution")

    class pixel:
        def __init__(self,found,value):
            self.found = found
            self.value = value

    #images = sitk.GetArrayFromImage(images)
    
    N = images.shape[0]  # Number of slices
    Y = images.shape[1]  # height of slices
    X = images.shape[2]  # width of slices
    start = 0
    #N=1400

    max_crack_heigth = 40
    max_crack_width = 40

    max_relative_thickness = 0.8

    I = (N-start)*X*Y
    
    skip_next_pixel = False
    print(f"we need to analyse {I} pixels. Across {N} images, {Y} rows, {X} columns.")
    
    #sweep accross all pixels
    print("starting primary sweep")
    i=0
    for n in range(start,N):                             
        loadingbar.main(i, I, bar_length=30)
        for x in range(0,X):
            skip_next_pixel = False
            for y in range(1,Y):
                i=i+1
                if images[n,y,x] != 0:
                    skip_next_pixel = False
                    continue
                if skip_next_pixel == True:
                    continue
                pixel_above = pixel(False, 0)
                pixel_below = pixel(False, 0)
                #print(f"working on image {n}, rows {y}, columns {x}.")
                for p in range(0, min(Y-y,y,max_crack_heigth)):             #look verticaly for the closest 2 pixels (1 above and 1 below)
                    skip_next_pixel = True
                    if pixel_above.found==False:
                        pixel_above.value = images[n,y+p,x]
                        if pixel_above.value!=0:
                            pixel_above.found = True
                    if pixel_below.found==False:
                        pixel_below.value = images[n,y-p,x]
                        if pixel_below.value!=0 :
                            pixel_below.found = True                        
                    
                    if pixel_above.found == True and pixel_below.found == True:
                        if pixel_below.value == pixel_above.value and pixel_above.value == 255:      #if you are in-between bone pixels
                            break                                                                       #this pixel is NOT in a crack
                        elif pixel_below.value == pixel_above.value and pixel_above.value == 99:    #if you are in-between keratine pixels
                            if p>10:                                                                        #but not too far appart
                                break
                            skip_next_pixel = False                                                     #this pixel is in a crack
                            images[n,y,x] = 128
                            break
                        elif pixel_below.value != pixel_above.value:                                #if you are in-between different pixels
                            skip_next_pixel = False                                                     #this pixel is in a crack
                            images[n,y,x] = 128
                            break

    print(' ')
    CT_visualization_functions.browse_images(sitk.GetImageFromArray(images))
    print("starting cleaning loop")
    i=0
    I_2 = (N-start)*X
    for n in range(start,N):        # this loop is for removing columns of crack pixels significantly longer than the horn above them
        #print(f"Working on image {n}...")
        loadingbar.main(i, I_2, bar_length=30)
        for x in range(0,X):
            column_pixel = np.array(images[n,:,x])
            i=i+1
            keratine_amount = np.count_nonzero(column_pixel == 99)
            crack_amount = np.count_nonzero(column_pixel == 128)

            if crack_amount > max_relative_thickness*keratine_amount:
                column_pixel[column_pixel==128] = 0
                images[n,:,x] = column_pixel
    
    
    CT_visualization_functions.browse_images(sitk.GetImageFromArray(images))
    
    print(" ")
    print("starting secondary sweep")
    i=0
    for n in range(start,N):
        #print(f"Working on image {n}...")
        loadingbar.main(i, I, bar_length=30)
        for x in range(0,X):
            skip_next_pixel = False                        #reset the skip pixel variable such that the first pixel in each column is not passed
            for y in range(0,Y):                           #with the three loop you go through all the pixels in the image stack
                i=i+1
                if images[n,y,x] != 0:                      #if pixel is non-zero, continue to the next pixel, but do not skip the next
                    skip_next_pixel = False
                    continue
                if skip_next_pixel == True:                 #if at the previous pixel, you defined that this pixel should be skipped, skip it
                    continue

                pixel_left = pixel(False, 0)    #setup variables
                pixel_right = pixel(False, 0)
                for k in range(0, min(X-x,x,max_crack_width)):          #look horizontaly around the main pixel, at 2 other pixels (1 left and 1 right), each iteration you look further away
                    skip_next_pixel = True                         #you start by assuming that the next pixel can be skipped
                    if pixel_left.found==False:                    #if the left pixel has not been found
                        pixel_left.value = images[n,y,x-k]
                        if pixel_left.value!=0:                          #check if this one could be it. aka is it not nul?
                            pixel_left.found = True                         #if yes, indicate that you found it
                            #print("left pixel found")

                    if pixel_right.found==False:                   #if the right pixel has not been found
                        pixel_right.value = images[n,y,x+k]
                        if pixel_right.value!=0:                          #check if this one could be it. aka is it not nul?
                            pixel_right.found = True                         #if yes, indicate that you found it
                            #print("right pixel found")
                        
                    if pixel_left.found == True and pixel_right.found == True:         #if both left and right pixels have been found
                        #print("looking at one pixel")
                        if pixel_right.value!=pixel_left.value and pixel_right.value!=128 and pixel_left.value!=128:  #if you are in between bone and keratine
                            images[n,y,x] = 128
                            skip_next_pixel = False                                           #do not skip next pixel
                            break
                        
                        elif pixel_right.value==pixel_left.value and pixel_right.value==255:          #if you are in between bone
                            break

                        elif pixel_right.value==pixel_left.value and pixel_right.value==255:          #if you are in between horn
                            if k>10:                                                                        #but not too far appart
                                break
                            images[n,y,x] = 128
                            skip_next_pixel = False                                             #do not skip next pixel
                            break
                        
                        elif pixel_right.value!=pixel_left.value and (pixel_right.value==128 or pixel_left.value==128):  #if you are in between crack and something else
                            if k>max_crack_width/3:
                                break
                            images[n,y,x] = 128
                            skip_next_pixel = False                                             #do not skip next pixel
                            break
                        
                        else:
                            break
    
    print(' ')
    CT_visualization_functions.browse_images(sitk.GetImageFromArray(images))
    print("starting secondary cleaning loop")
    i=0
    I_3 = (N-start)*Y
    for n in range(start,N):        # this loop is for removing columns of crack pixels significantly longer than the horn above them
        #print(f"Working on image {n}...")
        loadingbar.main(i, I_3, bar_length=30)
        for y in range(0,Y):
            line_pixel = np.array(images[n,y,:])
            i=i+1
            keratine_amount = np.count_nonzero(column_pixel == 99)
            crack_amount = np.count_nonzero(column_pixel == 128)

            if crack_amount > max_relative_thickness*keratine_amount:
                column_pixel[column_pixel==128] = 0
                images[n,y,:] = column_pixel
    
    print("stuff done")
    return images


convolved_image_array = detect_crack(images_array)

CT_visualization_functions.browse_images(sitk.GetImageFromArray(convolved_image_array))




starting 3D convolution
we need to analyse 1363912704 pixels. Across 1922 images, 693 rows, 1024 columns.
starting primary sweep
Progress: [#############################-] 99% 
starting cleaning loop
Progress: [------------------------------] 0% 
starting secondary sweep
Progress: [#############################-] 99% 
starting secondary cleaning loop
Progress: [------------------------------] 0%stuff done


In [ ]:
import cv2

images = sitk.ReadImage(direction_multimatrial_map)
images_array = np.array(sitk.GetArrayFromImage(images)).astype(np.uint8)


def detect_crack(images):
    "to whoever will have to understand this code: fuck you"
    print("starting 3D convolution")

    class pixel:
        def __init__(self,found,value):
            self.found = found
            self.value = value

    #images = sitk.GetArrayFromImage(images)
    
    N = images.shape[0]  # Number of slices
    Y = images.shape[1]  # height of slices
    X = images.shape[2]  # width of slices
    start = 0
    #N=1000

    max_crack_heigth = 60
    max_crack_width = 15
    max_crack_hole = 20

    max_relative_thickness = 1.5

    I = (N-start)*X*Y
    
    skip_next_pixel = False
    print(f"we need to analyse {I} pixels. Across {N} images, {Y} rows, {X} columns.")
    
    # =============================== first sweep ==============================
    # going through all the pixels column by column looking for pixels in between horn and bone
    # ==========================================================================
    print("starting vertical sweep")
    y_images = np.copy(images)
    i=0
    for n in range(start,N):                             
        loadingbar.main(i, I, bar_length=30)
        for x in range(0,X):
            if np.unique(y_images[n,:,x]).shape[0]<3:       #if there is only 2 colors (black and something else) in this line, ain't no way to find a crack ... HUGE speed boost
                i = i + y_images[n,:,x].shape[0]
                continue
            skip_next_pixel = False
            for y in range(0,Y):                           #with the three loop you go through all the pixels in the image stack

                i=i+1
                if y_images[n,y,x] != 0:                      #if pixel is non-zero, continue to the next pixel, but do not skip the next
                    skip_next_pixel = False
                    continue
                elif skip_next_pixel == True:                 #if at the previous pixel, you defined that this pixel should be skipped, skip it
                    continue
                elif y_images[n,y-1,x] == 0:                #if somehow the previous pixel is null and yet this one was not set to skip...
                    skip_next_pixel == True
                    continue
                pixel_above = pixel(True, y_images[n,y-1,x])    #setup variables. 
                pixel_below = pixel(False, 0)

                for p in range(0, min(Y-y,y,max_crack_heigth)):             #look verticaly for the closest 2 pixels (1 above and 1 below)
                    skip_next_pixel = True
                    
                    if pixel_below.found==False:
                        pixel_below.value = images[n,y+p,x]
                        if pixel_below.value!=0 :
                            pixel_below.found = True                        
                    
                    if pixel_above.found == True and pixel_below.found == True:
                        if pixel_below.value == pixel_above.value and pixel_above.value == 255:      #if you are in-between bone pixels
                            break                                                                       #this pixel is NOT in a crack
                        elif pixel_below.value == pixel_above.value and pixel_above.value == 99:    #if you are in-between keratine pixels
                            if p>10:                                                                        #but not too far appart
                                break
                            #skip_next_pixel = False                                                     #this pixel is in a crack
                            y_images[n,y:y+p,x] = 128
                            break
                        elif pixel_below.value != pixel_above.value:                                #if you are in-between different pixels
                            #skip_next_pixel = False                                                     #this pixel is in a crack
                            y_images[n,y:y+p,x] = 128
                            break
    CT_visualization_functions.browse_images(sitk.GetImageFromArray(y_images))



    # =============================== second sweep ==============================
    # going through all the pixels line by line looking for pixels in between horn and bone
    # ==========================================================================
    print(" ")
    print("starting horizontal sweep")
    x_images = np.copy(images)
    i=0
    for n in range(start,N):
        #print(f"Working on image {n}...")
        loadingbar.main(i, I, bar_length=30)
        for y in range(0,Y):
            if np.unique(x_images[n,y,:]).shape[0]<3:       #if there is only 2 colors (black and something else) in this line, ain't no way to find a crack ... HUGE speed boost
                i = i + y_images[n,y,:].shape[0]
                continue
            skip_next_pixel = False                        #reset the skip pixel variable such that the first pixel in each column is not passed
            for x in range(0,X):                           #with the three loop you go through all the pixels in the image stack
                i=i+1
                if x_images[n,y,x] != 0:                      #if pixel is non-zero, continue to the next pixel, but do not skip the next
                    skip_next_pixel = False
                    continue
                elif skip_next_pixel == True:                 #if at the previous pixel, you defined that this pixel should be skipped, skip it
                    continue
                elif x_images[n,y,x-1] == 0:                #if somehow the previous pixel is null and yet this one was not set to skip...
                    skip_next_pixel == True
                    continue

                pixel_left = pixel(True, x_images[n,y,x-1])    #setup variables. 
                pixel_right = pixel(False, 0)
                for k in range(0, min(X-x,x,max_crack_width)):          #look horizontaly around the main pixel, at 2 other pixels (1 left and 1 right), each iteration you look further away
                    skip_next_pixel = True                         #you start by assuming that the next pixel can be skipped

                    if pixel_right.found==False:                   #if the right pixel has not been found
                        pixel_right.value = images[n,y,x+k]
                        if pixel_right.value!=0:                          #check if this one could be it. aka is it not nul?
                            pixel_right.found = True                         #if yes, indicate that you found it
                            #print("right pixel found")
                        
                    if pixel_left.found == True and pixel_right.found == True:         #if both left and right pixels have been found
                        #print("looking at one pixel")
                        if pixel_right.value==pixel_left.value and pixel_right.value==255:          #if you are in between bone
                            break

                        elif pixel_right.value==pixel_left.value and pixel_right.value==99:          #if you are in between horn
                            if k>10:                                                                        #but not too far appart
                                break
                            x_images[n,y,x:x+k] = 128
                            #skip_next_pixel = False                                             #do not skip next pixel
                            break

                        elif pixel_right.value!=pixel_left.value :                          #if you are in between different pixels
                            x_images[n,y,x:x+k] = 128
                            #skip_next_pixel = False                                           #do not skip next pixel
                            break
    CT_visualization_functions.browse_images(sitk.GetImageFromArray(x_images))          

    images[x_images==128] = 128
    images[y_images==128] = 128
    CT_visualization_functions.browse_images(sitk.GetImageFromArray(images)) 


    # =============================== cleaning loop ==============================
    # removing columns where too many pixels were added
    # ==========================================================================
    print(' ')
    print("starting cleaning loop")
    I_2 = (N-start)*X
    i=0
    for n in range(start,N):
        #print(f"Working on image {n}...")
        loadingbar.main(i, I_2, bar_length=30)
        for x in range(0,X):
            column_pixel = np.array(images[n,:,x])
            i=i+1
            keratine_amount = np.count_nonzero(column_pixel == 99)
            crack_amount = np.count_nonzero(column_pixel == 128)
            if crack_amount > max_relative_thickness*keratine_amount:
                column_pixel[column_pixel==128] = 0
                images[n,:,x] = column_pixel

    CT_visualization_functions.browse_images(sitk.GetImageFromArray(images))


    # =============================== third sweep ==============================
    # going through all the pixels column by column looking for pixels in between previously found crack pixels
    # ==========================================================================
    print(" ")
    print("starting secondary vertical sweep")
    y_images = np.copy(images)
    i=0
    for n in range(start,N):                             
        loadingbar.main(i, I, bar_length=30)
        for x in range(0,X):
            if np.unique(y_images[n,:,x]).shape[0]<3:       #if there is only 2 colors (black and something else) in this line, ain't no way to find a crack ... HUGE speed boost
                i = i + y_images[n,:,x].shape[0]
                continue
            skip_next_pixel = False
            for y in range(0,Y):                           #with the three loop you go through all the pixels in the image stack
                i=i+1
                if y_images[n,y,x] != 0:                      #if pixel is non-zero, continue to the next pixel, but do not skip the next
                    skip_next_pixel = False
                    continue
                elif skip_next_pixel == True:                 #if at the previous pixel, you defined that this pixel should be skipped, skip it
                    continue
                elif y_images[n,y-1,x] == 0:                #if somehow the previous pixel is null and yet this one was not set to skip...
                    skip_next_pixel == True
                    continue
                pixel_above = pixel(True, y_images[n,y-1,x])    #setup variables. 
                pixel_below = pixel(False, 0)
                for p in range(0, min(Y-y,y,max_crack_hole)):          #look horizontaly around the main pixel, at 1 other pixels (right), each iteration you look further away
                    skip_next_pixel = True                         #you start by assuming that the next pixel can be skipped

                    if pixel_below.found==False:                   #if the right pixel has not been found
                        pixel_below.value = images[n,y+p,x]
                        if pixel_below.value!=0:                          #check if this one could be it. aka is it not null?
                            pixel_below.found = True                         #if yes, indicate that you found it
                            #print("right pixel found")

                    if pixel_above.found == True and pixel_below.found == True:
                        if pixel_above.value != 128 and pixel_below.value != 128:      #if you are not close to a crack pixel
                            break                                                                       #this pixel is NOT in a crack
                        elif pixel_above.value == pixel_below.value and pixel_below.value == 128:    #if you are in-between crack pixels
                            #skip_next_pixel = False                                                     #this pixel is in a crack
                            y_images[n,y:y+p,x] = 128
                            break
                        elif pixel_above.value != pixel_below.value and (pixel_above.value == 128 or pixel_below.value == 128):    #if you are in-between 1 crack pixel and another pixel
                            #skip_next_pixel = False                                                     #this pixel is in a crack
                            y_images[n,y:y+p,x] = 128
                            break
                        else:
                            break
    CT_visualization_functions.browse_images(sitk.GetImageFromArray(y_images))
    images[y_images==128] = 128

    # =============================== fourth sweep ==============================
    # going through all the pixels line by line looking for pixels in between previously found crack pixels
    # ==========================================================================
    print(" ")
    print("starting secondary horizontal sweep")
    x_images = np.copy(images)
    i=0
    for n in range(start,N):
        #print(f"Working on image {n}...")
        loadingbar.main(i, I, bar_length=30)
        for y in range(0,Y):
            if np.unique(x_images[n,y,:]).shape[0]<3:       #if there is only 2 colors (black and something else) in this line, ain't no way to find a crack ... HUGE speed boost
                i = i + y_images[n,y,:].shape[0]
                continue
            skip_next_pixel = False                        #reset the skip pixel variable such that the first pixel in each column is not passed
            for x in range(0,X):                           #with the three loop you go through all the pixels in the image stack
                i=i+1
                if x_images[n,y,x] != 0:                      #if pixel is non-zero, continue to the next pixel, but do not skip the next
                    skip_next_pixel = False
                    continue
                elif skip_next_pixel == True:                 #if at the previous pixel, you defined that this pixel should be skipped, skip it
                    continue
                elif x_images[n,y,x-1] == 0:                #if somehow the previous pixel is null and yet this one was not set to skip...
                    skip_next_pixel == True
                    continue
                pixel_left = pixel(True, x_images[n,y,x-1])    #setup variables. 
                pixel_right = pixel(False, 0)
                for k in range(0, min(X-x,x,max_crack_hole)):          #look horizontaly around the main pixel, at 1 other pixels (right), each iteration you look further away
                    skip_next_pixel = True                         #you start by assuming that the next pixel can be skipped
                    if pixel_right.found==False:                   #if the right pixel has not been found
                        pixel_right.value = images[n,y,x+k]
                        if pixel_right.value!=0:                          #check if this one could be it. aka is it not null?
                            pixel_right.found = True                         #if yes, indicate that you found it
                            #print("right pixel found")

                    if pixel_left.found == True and pixel_right.found == True:
                        if pixel_left.value != 128 and pixel_right.value != 128:      #if you are not close to a crack pixel
                            break                                                                       #this pixel is NOT in a crack
                        elif pixel_left.value == pixel_right.value and pixel_right.value == 128:    #if you are in-between crack pixels
                            #skip_next_pixel = False                                                     #this pixel is in a crack
                            x_images[n,y,x:x+k] = 128
                            break
                        elif pixel_left.value != pixel_right.value and (pixel_left.value == 128 or pixel_right.value == 128):    #if you are in-between 1 crack pixel and another pixel
                            #skip_next_pixel = False                                                     #this pixel is in a crack
                            x_images[n,y,x:x+k] = 128
                            break
                        else:
                            break
                    
                
    CT_visualization_functions.browse_images(sitk.GetImageFromArray(x_images))         
    images[x_images==128] = 128

    # =============================== fifth sweep ==============================
    # going through all the pixels n-diemsnion wise looking for pixels in between previously found crack pixels
    # ==========================================================================
    print(" ")
    print("starting secondary deep sweep")
    n_images = np.copy(images)
    i=0
    for x in range(0,X):
        #print(f"Working on image {n}...")
        loadingbar.main(i, I, bar_length=30)
        for y in range(0,Y):
            skip_next_pixel = False               #reset the skip pixel variable such that the first pixel in each column is not passed
            if np.unique(n_images[:,y,x]).shape[0]<3:       #if there is only 2 colors (black and something else) in this line, ain't no way to find a crack ... HUGE speed boost
                i = i + y_images[:,y,x].shape[0]
                continue
            for n in range(start,N):                           #with the three loop you go through all the pixels in the image stack
                i=i+1
                if n_images[n,y,x] != 0:                      #if pixel is non-zero, continue to the next pixel, but do not skip the next
                    skip_next_pixel = False
                    continue
                elif skip_next_pixel == True:                 #if at the previous pixel, you defined that this pixel should be skipped, skip it
                    continue
                elif n_images[n-1,y,x] == 0:                #if somehow the previous pixel is null and yet this one was not set to skip...
                    skip_next_pixel == True
                    continue
                pixel_left = pixel(True, n_images[n-1,y,x])    #setup variables. 
                pixel_right = pixel(False, 0)
                for l in range(0, min(N-n,n,max_crack_hole)):          #look horizontaly around the main pixel, at 1 other pixels (right), each iteration you look further away
                    skip_next_pixel = True                         #you start by assuming that the next pixel can be skipped

                    if pixel_right.found==False:                   #if the right pixel has not been found
                        pixel_right.value = images[n+l,y,x]
                        if pixel_right.value!=0:                          #check if this one could be it. aka is it not null?
                            pixel_right.found = True                         #if yes, indicate that you found it
                            #print("right pixel found")

                    if pixel_left.found == True and pixel_right.found == True:
                        if pixel_left.value != 128 and pixel_right.value != 128:      #if you are not close to a crack pixel
                            break                                                                       #this pixel is NOT in a crack
                        elif pixel_left.value == pixel_right.value and pixel_right.value == 128:    #if you are in-between crack pixels
                            #skip_next_pixel = True                                                     #this pixel is in a crack
                            n_images[n:n+l,y,x] = 128
                            break
                        elif pixel_left.value != pixel_right.value and (pixel_left.value == 128 or pixel_right.value == 128):    #if you are in-between 1 crack pixel and another pixel
                            #skip_next_pixel = True                                                     #this pixel is in a crack
                            n_images[n:n+l,y,x] = 128
                            break
                        else:
                            break
                        
    CT_visualization_functions.browse_images(sitk.GetImageFromArray(n_images))         
    images[n_images==128] = 128
    
    print("stuff done")
    return images


convolved_image_array = detect_crack(images_array)

CT_visualization_functions.browse_images(sitk.GetImageFromArray(convolved_image_array))




starting 3D convolution
we need to analyse 1363912704 pixels. Across 1922 images, 693 rows, 1024 columns.
starting vertical sweep
Progress: [############------------------] 42%

In [11]:
CT_visualization_window.open_window(sitk.GetImageFromArray(convolved_image_array))

2024-11-26 13:14:25,770 - CT_window_logger - INFO - starting window setup function
2024-11-26 13:14:53,963 - CT_window_logger - INFO - window setup function succesfull


array([], dtype=float64)

#===========================================================================================================================
#=================================================== manual mask ============================================================
#===========================================================================================================================


In [6]:
#make the mask

out = CT_visualization_window.open_window(sitk.GetImageFromArray(convolved_image_array))

#make sure that only zero pixel was selected
new_out = []
for row in out:
    if convolved_image_array[row[2],row[1],row[0]] == 0: #numpy array is a pain so numbers are reversed
        new_out.append(row)                              #only keep the row in out that reffer to zero pixel in the image
out = new_out


np.savetxt(output_folder_path + '/mask_crackfill.txt', out, fmt='%d', delimiter=',', header='x,y,slice', comments='', newline='\n')

2024-11-26 13:09:23,663 - CT_window_logger - INFO - starting window setup function
2024-11-26 13:11:08,536 - CT_window_logger - INFO - window setup function succesfull


In [7]:
#load and apply the mask

masked_image_array = np.copy(convolved_image_array)
manual_mask_path = output_folder_path + '/mask_crackfill.txt'

def load_data_as_numpy_array(file_path):
    # Read the data from the file
    data = pd.read_csv(file_path, sep=',', header=0)
    # Convert the list of tuples into a numpy array
    return np.array(data)


manual_mask_array = load_data_as_numpy_array(manual_mask_path)

for row in manual_mask_array:
    masked_image_array[row[2],row[1],row[0]] = 128 #numpy array is a pain so numbers are reversed
    
masked_image = sitk.GetImageFromArray(masked_image_array)
CT_visualization_functions.browse_images(masked_image)

In [8]:
#transform crack pixels into horn pixels
masked_image_array[masked_image_array == 128] = 99
masked_image_array = masked_image_array.astype(np.uint8)
masked_image = sitk.GetImageFromArray(masked_image_array)
CT_visualization_functions.browse_images(masked_image)

#===========================================================================================================================
#================================================ island analysis mask =======================================================
#===========================================================================================================================

In [10]:
horn_array = np.copy(masked_image_array)
horn_array[horn_array!=99] = 0

#island analysis
horn_image = sitk.GetImageFromArray(horn_array)
left_horn_image = CT_modification_functions.connected_filter(374,108,1183,horn_image)
right_horn_image = CT_modification_functions.connected_filter(673,116,1183,horn_image)
horn_array = sitk.GetArrayFromImage(left_horn_image) + sitk.GetArrayFromImage(right_horn_image)

#smoothing a bit
horn_image = sitk.GetImageFromArray(horn_array)
CT_modification_functions.dilate_filter(horn_image,1)
CT_modification_functions.erode_filter(horn_image,1)
CT_modification_functions.dilate_filter(horn_image,1)
CT_modification_functions.erode_filter(horn_image,1)

horn_array = sitk.GetArrayFromImage(horn_image)
horn_array[masked_image_array==255]=0

#apply it to the whole image
final_image_array = np.copy(masked_image_array)
final_image_array[final_image_array==99] = 0
final_image_array[horn_array!=0] = 99

final_image = sitk.GetImageFromArray(final_image_array)

CT_visualization_functions.browse_images(final_image)
CT_visualization_functions.browse_blended_images(final_image, masked_image,True)

#===========================================================================================================================
#================================================= save results ============================================================
#===========================================================================================================================

In [8]:
origin = images.GetOrigin()
spacing = images.GetSpacing()
direction = images.GetDirection()

final_image.SetSpacing(spacing)
final_image.SetOrigin(origin)
final_image.SetDirection(direction)

sitk.WriteImage(final_image, output_folder_path + "/map_multimaterial_crackfilled.mhd")